<br>

# Introdução


**Objetivo:**

1. Carregar os dados brutos dos arquivos que estão com extensão *.parquet*.
2. Trata-los, corrigindo dtypes, eliminando dados faltantes etc;
3. Fazer o *upload* em um banco de dados na *web*.

In [105]:
#!pip3 install dask --upgrade
#!pip3 install dask-labextension  --upgrade
#!pip3 install pyarrow --upgrade        # Necessário para usar o parquet
#!pip3 install traquitanas --upgrade
#!jupyter labextension install dask-labextension

In [66]:
import os
import sys
import time
import ctypes
import datetime
import numpy as np
import pandas as pd
import pyarrow.parquet as pq

In [67]:
from open_geodata import geo

In [68]:
import dask.dataframe as dd
from dask import compute
from dask.delayed import delayed
from dask.distributed import Client, LocalCluster
from dask.distributed import wait, progress
from dask.diagnostics import ProgressBar

In [69]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Text, String, DateTime, Float
from sqlalchemy.sql import text

In [70]:
import dask
import dask.distributed  # populate config with distributed defaults
dask.config.get('distributed.client')

{'heartbeat': '5s',
 'scheduler-info-interval': '2s',
 'security-loader': None,
 'preload': [],
 'preload-argv': []}

In [71]:
mod_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
sys.path.append(mod_path)
from sisagua.ibge import *

In [72]:
from paths import *

<br>

## Database

In [73]:
sys.path.append(os.path.expanduser('~/Codes'))
from my_vault.credentials_bitio import credential

In [74]:
api = credential['api']
URI = 'postgresql://michelmetran_demo_db_connection:{}@db.bit.io?sslmode=prefer'.format(api)
engine = create_engine(URI, isolation_level='AUTOCOMMIT')

<br>

## Functions

In [75]:
import csv
from io import StringIO


# Custom insert method for DataFrame.to_sql
def psql_insert_copy(table, conn, keys, data_iter):
    """
    Execute SQL statement inserting data

    Parameters
    ----------
    table : pandas.io.sql.SQLTable
    conn : sqlalchemy.engine.Engine or sqlalchemy.engine.Connection
    keys : list of str
        Column names
    data_iter : Iterable that iterates the values to be inserted
    """
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join(f'"{k}"' for k in keys)
        table_name = f'"{table.schema}"."{table.name}"'
        sql = f'COPY {table_name} ({columns}) FROM STDIN WITH CSV'
        cur.copy_expert(sql=sql, file=s_buf)

In [76]:
def rename_sisagua(df):
    df.rename(
        {
            'Cep': 'cep',
            'Endereço': 'endereco',
            'Número': 'numero',
            'Ddd': 'ddd',
            'Telefone': 'telefone',
            'Email': 'email',
            'Tempo Médio Diário De Funcionamento': 'tempo_medio_diario_funcionamento',
            'Etapa Pré-Oxidação': 'etapa_preoxidacao',
            'Etapa Mistura Rápida/Coagulação': 'etapa_mistura_rapida_coagulacao',
            'Etapa Floculação': 'etapa_floculacao',
            'Etapa Decantação': 'etapa_decantacao',
            'Etapa Flotação': 'etapa_flotacao',
            'Tipo De Filtração':'tipo_filtracao',
            'Número De Filtros': 'n_filtros',
            
            'Etapa Desinfecção': 'etapa_desinfeccao',
            'Desinfecção Com Cloro  Gás Ou Hipoclorito': 'desinfeccao_cloro_gas_ou_hipoclorito',
            'Desinfecção Com Cloramina':'desinfeccao_cloramina',
            'Desinfecção Com Dióxido Cloro': 'desinfeccao_dioxido_cloro',
            'Desinfecção Com Ozônio': 'desinfeccao_ozonio',
            'Desinfecção Com Uv': 'desinfeccao_uv',
            'Outro Agente Desinfetante':'outro_agente_desinfetante',
            
            'Rad Cloro Residual Livre': 'rad_cloro_residual_livre',
            'Rad Cloro Residual Combinado': 'rad_cloro_residual_combinado',
            'Rad Dióxido De Cloro': 'rad_dioxido_cloro',
            'Etapa Fluoretação': 'etapa_fluoretacao',
            'Outra Etapa De Tratamento': 'outra_etapa_tratamento',
            'Vazão De Água Tratada': 'vazao_agua_tratada',

            'Responsável Técnico': 'resp_tec_nome',
            'Formação Profissional Do Responsável Técnico': 'resp_tec_formacao',
            'Número Do Conselho De Classe Do Responsável Técnico': 'resp_tec_n_conselho_classe',
            'Anotação De Responsabilidade Técnica (Art)': 'art', 

            'Carro Pipa': 'carro_pipa',
            'Chafariz': 'chafariz',
            'Fonte': 'fonte',
            'Cisterna': 'cisterna',
            'Canalização': 'canalizacao',
            "Caixa D'Água": 'caixa_dagua',
            'Sem Reservação': 'sem_reservacao',
            'Outro Tipo De Suprimento': 'outro_tipo_suprimento',
            'Captação Superficial': 'captacao_superficial',
            'Captação Subterrânea': 'captacao_subterranea',
            'Captação De Água De Chuva': 'captacao_agua_chuva',
            'Filtração': 'filtracao',
            'Desinfecção': 'desinfeccao',
            
            'Número De Economias Residenciais (Domicílios Permanentes)': 'n_economias_residenciais_uso_permanente',
            'Número De Economias Residenciais (De Uso Ocasional)': 'n_economias_residenciais_uso_ocasional',
            
            'Razão Habitantes/Domicílio': 'razao_habitantes_domicilio',
            'Pop Recebe Água De Saa': 'pop_recebe_agua_saa',
            'Pop Recebe Água De Saa/Sac': 'pop_recebe_agua_saa_sac',
            
            'Região Geográfica': 'regiao_geografica',
            'Regional De Saúde': 'regional_saude',
            'Código Ibge': 'id_ibge',
            'Município':'municipio',
            'Uf': 'uf',
            
            'Tipo De Captação': 'captacao_tipo',
            'Categoria Do Manancial Superficial': 'manancial_superficial_categoria',
            'Nome Do Manancial Superficial': 'manancial_superficial_nome',
            'Categoria Do Ponto De Captação Subterrânea': 'captacao_subterranea_categoria',
            'Categoria Do Ponto De Captação Subterrâneo': 'captacao_subterranea_categoria',
            'Nome Do Ponto De Captação Subterrâneo': 'captacao_subterranea_nome',
            'Nome Do Ponto De Captação Subterrânea': 'captacao_subterranea_nome',
            'Outorga': 'outorga',
            
            'Tipo Da Instituição': 'instituicao_tipo',
            'Sigla Da Instituição':'instituicao_sigla',
            'Nome Da Instituição': 'instituicao_nome',
            'Nome Da Instiuição': 'instituicao_nome',
            'Cnpj Da Instituição': 'instituicao_cnpj',
            
            'Nome Do Escritório Regional/Local': 'escritorio_regional_local_nome',
            'Cnpj Do Escritório Regional/Local': 'escritorio_regional_local_cnpj',
            
            'Tipo Da Forma De Abastecimento': 'forma_abastecimento_tipo',
            'Código Forma De Abastecimento': 'forma_abastecimento_cod',            
            'Nome Da Forma De Abastecimento': 'forma_abastecimento_nome',
            
            'Ano De Referência': 'ano_referencia',
            'Ano': 'ano_referencia',
            'Semestre De Referência': 'semestre_referencia',
            'Mês De Referência': 'mes_referencia',
            'Mês': 'mes_referencia',
            
            'Data Da Coleta': 'data_coleta',
            'Data Da Análise': 'data_analise',
            'Data Do Laudo': 'data_laudo',
            'Data De Registro No Sisagua': 'data_registro_sisagua',
            'Data De Preenchimento': 'data_preenchimento',            
            'Data De Registro': 'data_registro',
            'Data De Preenchimento Do Relatório Mensal': 'data_preenchimento_rel_mensal',
            'Data De Preenchimento Do Relatório Semestral': 'data_preenchimento_rel_semestral',
            
            'Zona': 'zona',
            'Categoria Área': 'area_categoria',
            'Área': 'area_nome',
            'Descrição Do Local': 'local_descricao',
            'Tipo Do Local': 'local_tipo',
            'Local': 'local_nome',
            'Motivo Da Coleta': 'coleta_motivo',
            'Procedência Da Coleta': 'coleta_procedencia',
            'Ponto De Coleta': 'coleta_ponto',
            'Latitude': 'latitude',
            'Longitude': 'longitude',

            'Nome Da Eta/Uta': 'eta_uta_nome',
            'Nome Da Eta / Uta': 'eta_uta_nome',
            'Vazão': 'vazao',
            
            'Número Da Amostra': 'n_amostra',
            'Número De Reparos Na Rede (Somente Para Saa)': 'n_reparos_rede',
            'Número De Eventos De Intermitência (Somente Para Saa)': 'n_eventos_intermitencia',
            'Número De Eventos De Falta De Água': 'n_eventos_falta_agua',
            'Número De Reclamações De Cor Da Água': 'n_reclamacao_cor',
            'Número De Reclamação De Gosto E Ou Odor': 'n_reclamacao_gosto_odor',
            'Ponto De Monitoramento': 'pto_monitoramento',

            'Grupo': 'grupo',
            'Grupo De Parâmetros': 'grupo_parametros',
            'Parâmetro': 'parametro',
            'Parâmetro (Ciano)': 'parametro',
            'Parâmetro (Parâmetros Básicos)': 'parametro',
            'Parâmetro (Demais Parâmetros)': 'parametro',
            'Campo': 'campo',
            
            'Ld': 'lim_deteccao',
            'Lq': 'lim_quantificacao',
            'Resultado': 'resultado',
            'Valor': 'resultado',
            
            'Análise Realizada': 'analise_realizada',
            'Providência': 'providencia',
            'Providência Do Controle': 'providencia',
            'Unidade': 'unidade',
        },
        axis=1,
        inplace=True,
    )
    return df

<br>

## Client

In [77]:
import multiprocessing as mp
from dask.distributed import Client, LocalCluster

In [78]:
#import close_process
#close_process.process()

In [79]:
cluster = LocalCluster(
    n_workers=int(0.9 * mp.cpu_count()),
    threads_per_worker=16,
    processes=True,
    memory_limit='20GB',
    env={'MALLOC_TRIM_THRESHOLD_': '65536'}
)

client = Client(cluster)
client

/home/michel/Documents/conda/envs/pablocarreira-py39/lib/python3.9/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41437 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:41437/status,
Dashboard: http://127.0.0.1:41437/status,Workers: 7
Total threads: 112,Total memory: 130.39 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42911,Workers: 7
Dashboard: http://127.0.0.1:41437/status,Total threads: 112
Started: Just now,Total memory: 130.39 GiB
Comm: tcp://127.0.0.1:36453,Total threads: 16
Dashboard: http://127.0.0.1:39219/status,Memory: 18.63 GiB
Nanny: tcp://127.0.0.1:45319,


<br>

# Parâmetros

In [80]:
estado = 'SP'
cod_ibge_ajustado = '*'

<br>

# Cadastro

## Pontos Captação

In [ ]:
# Input
filename = os.path.join(input_path_parquet_partitioned, 'cadastro', 'cadastro_pontos_captacao')

# Add Filter to Filename
filter_path = os.path.join('Uf={}'.format(estado), 'Código Ibge={}/*.parquet'.format(cod_ibge_ajustado))
filename = os.path.join(filename, filter_path)

# Read Dataframes
df = dd.read_parquet(
    filename,
    filters=[[('Uf', '==', estado)]],
)

# Calculate
df_1 = df.compute()
df_1 = rename_sisagua(df_1)
print(df_1.info())
df_1.head()

In [ ]:
list(df_1.columns)

In [ ]:
df_1.drop(
    [
        # 'regiao_geografica',
        # 'regional_saude',
        # 'municipio',
        # 'instituicao_tipo',
        # 'instituicao_sigla',
        # 'instituicao_nome',
        # 'escritorio_regional_local_nome',
        # 'escritorio_regional_local_cnpj',
        # 'forma_abastecimento_tipo',
        # 'forma_abastecimento_cod',
        # 'forma_abastecimento_nome',
        # 'eta_uta_nome',
        # 'ano_referencia',
        # 'captacao_tipo',
        # 'manancial_superficial_categoria',
        # 'manancial_superficial_nome',
        # 'captacao_subterranea_categoria',
        # 'captacao_subterranea_nome',
        # 'outorga',
        # 'latitude',
        # 'longitude',
        # 'vazao',
        'uf',
        # 'id_ibge'
    ],
    inplace=True,
    axis=1,
    errors='ignore',
)

In [ ]:
df_1['id_ibge'] = df_1['id_ibge'].astype(int)

In [ ]:
df_1['ano_referencia'] = df_1['ano_referencia'].astype(int)
set(df_1['ano_referencia'])

In [ ]:
df_1['captacao_tipo'] = df_1['captacao_tipo'].str.title()
df_1.replace({'captacao_tipo': {'Subterraneo': 'Subterrânea'}}, inplace=True)
set(df_1['captacao_tipo'])

In [ ]:
df_1['manancial_superficial_categoria'] = df_1['manancial_superficial_categoria'].str.title()
set(df_1['manancial_superficial_categoria'])

In [ ]:
df_1['captacao_subterranea_categoria'] = df_1['captacao_subterranea_categoria'].str.title()
set(df_1['captacao_subterranea_categoria'])

In [ ]:
df_1.replace({'outorga': {'N': 'Não', 'S': 'Sim'}}, inplace=True)
set(df_1['outorga'])

In [ ]:
df_1['latitude'] = df_1['latitude'].astype(str).str.replace(',', '.')
df_1['latitude'] = pd.to_numeric(df_1['latitude'], errors='coerce')

In [ ]:
df_1['longitude'] = df_1['longitude'].astype(str).str.replace(',', '.')
df_1['longitude'] = pd.to_numeric(df_1['longitude'], errors='coerce')

In [ ]:
df_1['vazao'] = pd.to_numeric(df_1['vazao'])

In [ ]:
df_1.info()
df_1.head()

<br>

### Municípios

In [ ]:
list_cols = [
    'id_ibge',
    'municipio',
    'regional_saude',
]

df_1_1 = df_1[list_cols].drop_duplicates()
df_1_1.reset_index(drop=True, inplace=True)
df_1_1.info()
df_1_1.head()

<br>

### Instituições e Formas de Abastecimento

In [ ]:
list_cols = [
    'forma_abastecimento_cod',
    'forma_abastecimento_tipo',
    'forma_abastecimento_nome',
    'instituicao_tipo',
    'instituicao_sigla',
    'instituicao_nome',
    #'instituicao_cnpj',
    'escritorio_regional_local_nome',
    'escritorio_regional_local_cnpj',
    'eta_uta_nome',
    'ano_referencia',
    'id_ibge',
]
df_1_2 = df_1[list_cols].drop_duplicates()
df_1_2 = convert_6d_to_7d(df_1_2)
df_1_2 = df_1_2[list_cols].drop_duplicates()
df_1_2.reset_index(drop=True, inplace=True)
df_1_2.info()
df_1_2.head()

In [ ]:
len(list(set(df_1['forma_abastecimento_cod'])))

<br>

### Captação

In [ ]:
list_cols = [
    'forma_abastecimento_cod',
    
    'ano_referencia',
    'captacao_tipo',
    'manancial_superficial_categoria',
    'manancial_superficial_nome',
    'captacao_subterranea_categoria',
    'captacao_subterranea_nome',
    'outorga',
    'latitude',
    'longitude',
    'vazao',
    'id_ibge'
]
df_1_3 = df_1[list_cols].drop_duplicates()
df_1_3 = convert_6d_to_7d(df_1_3)
df_1_3 = df_1_3[list_cols].drop_duplicates()
df_1_3.reset_index(drop=True, inplace=True)
df_1_3.info()
df_1_3.head()

<br>

## População Abastecida

In [ ]:
# Input
filename = os.path.join(input_path_parquet_partitioned, 'cadastro', 'cadastro_populacao_abastecida')

# Add Filter to Filename
filter_path = os.path.join('Uf={}'.format(estado), 'Código Ibge={}/*.parquet'.format(cod_ibge_ajustado))
filename = os.path.join(filename, filter_path)

# Read Dataframes
df = dd.read_parquet(
    filename,
    filters=[[('Uf', '==', estado)]],
)

# Calculate
df_2 = df.compute()
df_2 = rename_sisagua(df_2)
print(df_2.info())
df_2.head()

In [ ]:
list(df_2.columns)

In [ ]:
df_2.drop(
    [
        # 'regiao_geografica',
        # 'regional_saude',
        # 'municipio',
        # 'instituicao_tipo',
        # 'instituicao_sigla',
        # 'instituicao_nome',
        # 'instituicao_cnpj',
        # 'escritorio_regional_local_nome',
        # 'escritorio_regional_local_cnpj',
        # 'forma_abastecimento_tipo',
        # 'forma_abastecimento_cod',
        # 'forma_abastecimento_nome',
        
        # 'ano_referencia',
        # 'data_registro',
        # 'data_preenchimento',
        # 'carro_pipa',
        # 'chafariz',
        # 'fonte',
        # 'cisterna',
        # 'canalizacao',
        # 'caixa_dagua',
        # 'sem_reservacao',
        # 'outro_tipo_suprimento',
        # 'captacao_superficial',
        # 'captacao_subterranea',
        # 'captacao_agua_chuva',
        # 'filtracao',
        # 'desinfeccao',
        # 'n_economias_residenciais_uso_permanente',
        # 'n_economias_residenciais_uso_ocasional',
        # 'razao_habitantes_domicilio',
        # 'pop_recebe_agua_saa',
        # 'pop_recebe_agua_saa_sac',
        'uf',
        # 'id_ibge'
    ],
    inplace=True,
    axis=1,
    errors='ignore',
)

In [ ]:
df_2['id_ibge'] = df_2['id_ibge'].astype(int)

In [ ]:
df_2['ano_referencia'] = df_2['ano_referencia'].astype(int)
set(df_2['ano_referencia'])

In [ ]:
df_2['data_registro'] = pd.to_datetime(df_2['data_registro'])
df_2['data_preenchimento'] = pd.to_datetime(df_2['data_preenchimento'])

In [ ]:
df_2['outro_tipo_suprimento'] = df_2['outro_tipo_suprimento'].str.title()

df_2.replace(
    {
        'outro_tipo_suprimento': {
            'Agua': 'Água',
            'Caixa D Água': "Caixa d'Água",
            "Caixa D' Água": "Caixa d'Água",
            'Suterranea': 'Subterrânea',
            'Subterraneo': 'Subterrâneo',
            'Subterranea': 'Subterrânea',
            'Galao': 'Galão',
            "Caixa D'Água": "Caixa d'Água",
            'Concessionaria': 'Concessionária',
            'Conessionaria': 'Concessionária',
            'Poco': 'Poço',
            'Poç0O': 'Poço',
            'Poso': 'Poço',
            'Tubulaçao': 'Tubulação',
            'Caminhao Pipa': 'Caminhão Pipa',
            'Caminho Pipa': 'Caminhão Pipa',
            'Ptrofundo': 'Profundo',
            'Profundio': 'Profundo',
            'Profubndo': 'Profundo',
            'Publica': 'Pública',
            'Pública1': 'Pública',
            'Reservatorio': 'Reservatório',
            'Rece': 'Rede',
            'Sist. Público': 'Sistema Público',
            'Publico': 'Público',
            'Astecimento': 'Abastecimento',
            'Poçotubular Profundo': 'Poço Tubular Profundo',            
            'N.A': 'Não Aplicável',
            'Nao Aplicave': 'Não Aplicável',
            'Não Aplicável': 'Não Aplicável',
            'Não Há': 'Não Aplicável',
            'Não Se Aplica': 'Não Aplicável',
            'Não Utiliza': 'Não Aplicável',            
            'Artesianio': 'Artesiano',
            'Artesisano': 'Artesiano',
            'Pública-Saae': 'Pública Saae',
        }
    },
    inplace=True,
    regex=True,
)
set(df_2['outro_tipo_suprimento'])

In [ ]:
df_2['n_economias_residenciais_uso_permanente'] = df_2['n_economias_residenciais_uso_permanente'].astype(float)
df_2['n_economias_residenciais_uso_ocasional'] = df_2['n_economias_residenciais_uso_ocasional'].astype(float)
df_2['razao_habitantes_domicilio'] = df_2['razao_habitantes_domicilio'].astype(float)
#set(df_2['n_economias_residenciais_uso_permanente'])

In [ ]:
df_2.replace({'pop_recebe_agua_saa': {'N': 'Não', 'S': 'Sim'}}, inplace=True)

In [ ]:
set(df_2['carro_pipa'])

<br>

### Municípios

In [ ]:
list_cols = [
    'id_ibge',
    'municipio',
    'regional_saude',
]

df_2_1 = df_2[list_cols].drop_duplicates()
df_2_1.reset_index(drop=True, inplace=True)
df_2_1.info()
df_2_1

<br>

### Instituições e Formas de Abastecimento

In [ ]:
list_cols = [
    'forma_abastecimento_cod',
    'forma_abastecimento_tipo',
    'forma_abastecimento_nome',
    'instituicao_tipo',
    'instituicao_sigla',
    'instituicao_nome',
    'instituicao_cnpj',
    'escritorio_regional_local_nome',
    'escritorio_regional_local_cnpj',
    #'eta_uta_nome',
    'ano_referencia',    
    'id_ibge',
]
df_2_2 = df_2[list_cols].drop_duplicates()
df_2_2 = convert_6d_to_7d(df_2_2)
df_2_2 = df_2_2[list_cols].drop_duplicates()
df_2_2.reset_index(drop=True, inplace=True)
df_2_2.info()
df_2_2.head()

<br>

### População

In [ ]:
list_cols = [
    'forma_abastecimento_cod',    
    'ano_referencia',
    'data_registro',
    'data_preenchimento',
    'carro_pipa',
    'chafariz',
    'fonte',
    'cisterna',
    'canalizacao',
    'caixa_dagua',
    'sem_reservacao',
    'outro_tipo_suprimento',
    'captacao_superficial',
    'captacao_subterranea',
    'captacao_agua_chuva',
    'filtracao',
    'desinfeccao',
    'n_economias_residenciais_uso_permanente',
    'n_economias_residenciais_uso_ocasional',
    'razao_habitantes_domicilio',
    'pop_recebe_agua_saa',
    'pop_recebe_agua_saa_sac',
    'id_ibge'
]
df_2_3 = df_2[list_cols].drop_duplicates()
df_2_3 = convert_6d_to_7d(df_2_3)
df_2_3 = df_2_3[list_cols].drop_duplicates()
df_2_3.reset_index(drop=True, inplace=True)
df_2_3.info()
df_2_3.head()

<br>

## Tratamento de Água

In [ ]:
# Input
filename = os.path.join(input_path_parquet_partitioned, 'cadastro', 'cadastro_tratamento_de_agua')

# Add Filter to Filename
filter_path = os.path.join('Uf={}'.format(estado), 'Código Ibge={}/*.parquet'.format(cod_ibge_ajustado))
filename = os.path.join(filename, filter_path)

# Read Dataframes
df = dd.read_parquet(
    filename,
    filters=[[('Uf', '==', estado)]],
)

# Calculate
df_3 = df.compute()
df_3 = rename_sisagua(df_3)
df_3.info()
df_3.head()

In [ ]:
list(df_3.columns)

In [ ]:
df_3.drop(
    [
        'regiao_geografica',
        # 'regional_saude',
        # 'municipio',
        # 'instituicao_tipo',
        # 'instituicao_sigla',
        # 'instituicao_nome',
        # 'instituicao_cnpj',
        # 'escritorio_regional_local_nome',
        # 'escritorio_regional_local_cnpj',
        # 'forma_abastecimento_tipo',
        # 'forma_abastecimento_cod',
        # 'forma_abastecimento_nome',
        # 'eta_uta_nome',
        # 'ano_referencia',
        # 'data_registro',
        # 'data_preenchimento',
        # 'captacao_superficial',
        # 'captacao_subterranea',
        # 'captacao_agua_chuva',
        # 'cep',
        # 'endereco',
        # 'numero',
        # 'ddd',
        # 'telefone',
        # 'email',
        # 'tempo_medio_diario_funcionamento',
        # 'etapa_preoxidacao',
        # 'etapa_mistura_rapida_coagulacao',
        # 'etapa_floculacao',
        # 'etapa_decantacao',
        # 'etapa_flotacao',
        # 'tipo_filtracao',
        # 'n_filtros',
        # 'etapa_desinfeccao',
        # 'desinfeccao_cloro_gas_ou_hipoclorito',
        # 'desinfeccao_cloramina',
        # 'desinfeccao_dioxido_cloro',
        # 'desinfeccao_ozonio',
        # 'desinfeccao_uv',
        # 'outro_agente_desinfetante',
        # 'rad_cloro_residual_livre',
        # 'rad_cloro_residual_combinado',
        # 'rad_dioxido_cloro',
        # 'etapa_fluoretacao',
        # 'outra_etapa_tratamento',
        # 'vazao_agua_tratada',
        # 'carro_pipa',
        # 'chafariz',
        # 'fonte',
        # 'cisterna',
        # 'canalizacao',
        # 'outro_tipo_suprimento',
        # 'resp_tec_nome',
        # 'resp_tec_formacao',
        # 'resp_tec_n_conselho_classe',
        # 'art',
        'uf',
        # 'id_ibge'
    ],
    inplace=True,
    axis=1,
    errors='ignore',
)

In [ ]:
df_3['id_ibge'] = df_3['id_ibge'].astype(int)

In [ ]:
df_3['ano_referencia'] = df_3['ano_referencia'].astype(int)
set(df_3['ano_referencia'])

In [ ]:
df_3['data_registro'] = pd.to_datetime(df_3['data_registro'], format='%d/%m/%Y')
df_3['data_preenchimento'] = pd.to_datetime(df_3['data_preenchimento'], format='%d/%m/%Y')

In [ ]:
#set(df_3['data_registro'])

In [ ]:
df_3['n_filtros'].replace(np.nan, -999, inplace=True)
df_3['n_filtros'] = df_3['n_filtros'].astype(int)
#set(df_3['n_filtros'])

In [ ]:
df_3['tipo_filtracao'] = df_3['tipo_filtracao'].str.title()

In [ ]:
df_3['vazao_agua_tratada'] = df_3['vazao_agua_tratada'].astype(float)
#set(df_3['vazao_agua_tratada'])

In [ ]:
list_cols = [
    'etapa_preoxidacao',
    'etapa_mistura_rapida_coagulacao',
    'etapa_floculacao',
    'etapa_decantacao',
    'etapa_flotacao',
    'etapa_desinfeccao',
    'desinfeccao_cloro_gas_ou_hipoclorito',
    'desinfeccao_cloramina',
    'desinfeccao_dioxido_cloro',
    'desinfeccao_ozonio',
    'desinfeccao_uv',
    'rad_cloro_residual_livre',
    'rad_cloro_residual_combinado',
    'rad_dioxido_cloro',
    'etapa_fluoretacao',
    'carro_pipa',
    'chafariz',
    'fonte',
    'cisterna',
    'canalizacao',
]
for col in list_cols:
    print(col)
    df_3.replace({col: {'N': 'Não', 'S': 'Sim'}}, regex=True, inplace=True)

In [ ]:
df_3['resp_tec_nome'] = df_3['resp_tec_nome'].str.title()
#set(df_3['resp_tec_nome'])

In [ ]:
#pd.to_datetime(df_3['tempo_medio_diario_funcionamento'], format='hh:mm')
set(df_3['tempo_medio_diario_funcionamento'])
#df_3['tempo_medio_diario_funcionamento'

In [ ]:
for i in list(df_3.columns)[44:]:
    #print(i)
    #print(set(df_3_3[i]))
    #print('  ')
    pass

In [ ]:
df_3.info()
df_3.head()

<br>

### Municipios

In [ ]:
list_cols = [
    'id_ibge',
    'municipio',
    'regional_saude',
]

df_3_1 = df_3[list_cols].drop_duplicates()
df_3_1.reset_index(drop=True, inplace=True)
df_3_1.info()
df_3_1.head()

<br>

### Instituições e Formas de Abastecimento

In [ ]:
list_cols = [
    'forma_abastecimento_cod',
    'forma_abastecimento_tipo',
    'forma_abastecimento_nome',
    'instituicao_tipo',
    'instituicao_sigla',
    'instituicao_nome',
    'instituicao_cnpj',
    'escritorio_regional_local_nome',
    'escritorio_regional_local_cnpj',
    'eta_uta_nome',
    'ano_referencia',
    'id_ibge',
]
df_3_2 = df_3[list_cols].drop_duplicates()
df_3_2 = convert_6d_to_7d(df_3_2)
df_3_2 = df_3_2[list_cols].drop_duplicates()
df_3_2.reset_index(drop=True, inplace=True)
df_3_2.info()
df_3_2.head()

<br>

### Tratamento

In [ ]:
list_cols = [
    'forma_abastecimento_cod',

    'ano_referencia',
    'data_registro',
    'data_preenchimento',
    'captacao_superficial',
    'captacao_subterranea',
    'captacao_agua_chuva',
    'cep',
    'endereco',
    'numero',
    'ddd',
    'telefone',
    'email',
    'tempo_medio_diario_funcionamento',
    'etapa_preoxidacao',
    'etapa_mistura_rapida_coagulacao',
    'etapa_floculacao',
    'etapa_decantacao',
    'etapa_flotacao',
    'tipo_filtracao',
    'n_filtros',
    'etapa_desinfeccao',
    'desinfeccao_cloro_gas_ou_hipoclorito',
    'desinfeccao_cloramina',
    'desinfeccao_dioxido_cloro',
    'desinfeccao_ozonio',
    'desinfeccao_uv',
    'outro_agente_desinfetante',
    'rad_cloro_residual_livre',
    'rad_cloro_residual_combinado',
    'rad_dioxido_cloro',
    'etapa_fluoretacao',
    'outra_etapa_tratamento',
    'vazao_agua_tratada',
    'carro_pipa',
    'chafariz',
    'fonte',
    'cisterna',
    'canalizacao',
    'outro_tipo_suprimento',
    'resp_tec_nome',
    'resp_tec_formacao',
    'resp_tec_n_conselho_classe',
    'art',
    'id_ibge'
]
df_3_3 = df_3[list_cols].drop_duplicates()
df_3_3 = convert_6d_to_7d(df_3_3)
df_3_3 = df_3_3[list_cols].drop_duplicates()
df_3_3.reset_index(drop=True, inplace=True)
df_3_3.info()
df_3_3.head()

<br>

# Vigilância

<br>

## Parâmetros Básicos

In [81]:
# Input
filename = os.path.join(input_path_parquet_partitioned, 'vigilancia', 'vigilancia_parametros_basicos_*')

# Add Filter to Filename
filter_path = os.path.join('Uf={}'.format(estado), 'Código Ibge={}/*.parquet'.format(cod_ibge_ajustado))
filename = os.path.join(filename, filter_path)

# Read Dataframes
df = dd.read_parquet(
    filename,
    filters=[[('Uf', '==', estado)]],
)

# Calculate
df_4 = df.compute()
df_4 = rename_sisagua(df_4)
df_4.info()
df_4.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3215727 entries, 0 to 114
Data columns (total 36 columns):
 #   Column                          Dtype   
---  ------                          -----   
 0   regiao_geografica               object  
 1   regional_saude                  object  
 2   municipio                       object  
 3   n_amostra                       object  
 4   coleta_motivo                   object  
 5   forma_abastecimento_tipo        object  
 6   forma_abastecimento_cod         object  
 7   forma_abastecimento_nome        object  
 8   instituicao_tipo                object  
 9   instituicao_sigla               object  
 10  instituicao_nome                object  
 11  instituicao_cnpj                object  
 12  escritorio_regional_local_nome  object  
 13  escritorio_regional_local_cnpj  object  
 14  eta_uta_nome                    object  
 15  ano_referencia                  object  
 16  mes_referencia                  object  
 17  data_coleta 

,regiao_geografica,regional_saude,municipio,n_amostra,coleta_motivo,forma_abastecimento_tipo,forma_abastecimento_cod,forma_abastecimento_nome,instituicao_tipo,instituicao_sigla,...,local_tipo,local_nome,latitude,longitude,parametro,analise_realizada,resultado,providencia,uf,id_ibge
0,SUDESTE,GVS XIX - MARÍLIA,ADAMANTINA,2160,Rotina,SAA,S350010000001,ADAMANTINA,Empresa Estadual,SABESP,...,None,None,None,None,Cor (uH),None,"2,00",None,SP,350010
1,SUDESTE,GVS XIX - MARÍLIA,ADAMANTINA,1424,Rotina,SAA,S350010000001,ADAMANTINA,Empresa Estadual,SABESP,...,None,None,None,None,pH,None,"7,60",None,SP,350010
2,SUDESTE,GVS XIX - MARÍLIA,ADAMANTINA,3740,Rotina,SAA,S350010000001,ADAMANTINA,Empresa Estadual,SABESP,...,None,None,None,None,Cloro residual livre (mg/L),EM_CAMPO,"0,80",None,SP,350010
3,SUDESTE,GVS XIX - MARÍLIA,ADAMANTINA,3251,Rotina,SAA,S350010000001,ADAMANTINA,Empresa Estadual,SABESP,...,None,None,None,None,Turbidez (uT),EM_LABORATORIO,"0,30",None,SP,350010
4,SUDESTE,GVS XIX - MARÍLIA,ADAMANTINA,1423,Rotina,SAA,S350010000001,ADAMANTINA,Empresa Estadual,SABESP,...,None,None,None,None,Turbidez (uT),EM_LABORATORIO,"0,20",None,SP,350010


In [82]:
list(df_4.columns)

['regiao_geografica',
 'regional_saude',
 'municipio',
 'n_amostra',
 'coleta_motivo',
 'forma_abastecimento_tipo',
 'forma_abastecimento_cod',
 'forma_abastecimento_nome',
 'instituicao_tipo',
 'instituicao_sigla',
 'instituicao_nome',
 'instituicao_cnpj',
 'escritorio_regional_local_nome',
 'escritorio_regional_local_cnpj',
 'eta_uta_nome',
 'ano_referencia',
 'mes_referencia',
 'data_coleta',
 'data_laudo',
 'data_registro_sisagua',
 'coleta_procedencia',
 'coleta_ponto',
 'local_descricao',
 'zona',
 'area_categoria',
 'area_nome',
 'local_tipo',
 'local_nome',
 'latitude',
 'longitude',
 'parametro',
 'analise_realizada',
 'resultado',
 'providencia',
 'uf',
 'id_ibge']

In [83]:
df_4.drop(
    [
        'regiao_geografica',
        'regional_saude',
        'municipio',
        # 'n_amostra',
        # 'coleta_motivo',
        # 'forma_abastecimento_tipo', # Ok
        # 'forma_abastecimento_cod', # Ok
        # 'forma_abastecimento_nome', # Ok
        # 'instituicao_tipo', # Ok
        # 'instituicao_sigla', # Ok
        # 'instituicao_nome', # Ok
        # 'instituicao_cnpj', # Ok
        # 'escritorio_regional_local_nome', # Ok
        # 'escritorio_regional_local_cnpj', # Ok
        # 'eta_uta_nome', # Ok
        # 'ano_referencia', # Resolvido
        # 'mes_referencia', # Resolvido
        # 'data_coleta', # Resolvido
        # 'data_laudo', # TODO
        # 'data_registro_sisagua', # Resolvido
        # 'coleta_procedencia',
        # 'coleta_ponto',
        # 'local_descricao',
        # 'zona',
        # 'area_categoria',
        # 'area_nome',
        # 'local_tipo',
        # 'local_nome',
        # 'latitude',
        # 'longitude',
        # 'parametro',
        # 'analise_realizada',
        # 'resultado',
        # 'providencia',
        'uf',
        # 'id_ibge', # Ok
    ],
    inplace=True,
    axis=1,
    errors='ignore',
)

In [84]:
df_4['id_ibge'] = df_4['id_ibge'].astype(int)
df_4 = convert_6d_to_7d(df_4)

In [85]:
set(df_4['coleta_motivo'])
set(df_4['forma_abastecimento_tipo'])

{'SAA', 'SAC', 'SAI'}

In [86]:
df_4['ano_referencia'] = df_4['ano_referencia'].astype(int)
set(df_4['ano_referencia'])

{2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022}

In [87]:
df_4['mes_referencia'] = df_4['mes_referencia'].astype(int)
set(df_4['mes_referencia'])

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

In [88]:
df_4['data_coleta'] = pd.to_datetime(df_4['data_coleta'])
df_4['data_registro_sisagua'] = pd.to_datetime(df_4['data_registro_sisagua'])

In [89]:
# TODO: Corrigir erro: Out of bounds nanosecond timestamp: 8201-08-05 00:00:00
#df_4['data_laudo'] = pd.to_datetime(df_4['data_laudo'])

In [90]:
#set(df_4['local_descricao'])
#set(df_4['zona'])
#set(df_4['area_categoria'])
#set(df_4['local_tipo'])

In [91]:
df_4['latitude'] = df_4['latitude'].astype(str).str.replace(',', '.')
df_4['latitude'] = pd.to_numeric(df_4['latitude'], errors='coerce')

In [92]:
df_4['longitude'] = df_4['longitude'].astype(str).str.replace(',', '.')
df_4['longitude'] = pd.to_numeric(df_4['longitude'], errors='coerce')

In [93]:
df_4['coleta_procedencia'] = df_4['coleta_procedencia'].str.title()
list(set(df_4['coleta_procedencia']))

['Estação De Tratamento De Água',
 'Intra-Domiciliar / Intra-Predial',
 'Solução Alternativa',
 'Sistema De Distribuição',
 'Ponto De Captação(Água Superficial Ou Subterrânea)']

In [94]:
df_4['area_nome'] = df_4['area_nome'].str.title()
#list(set(df_4['area_nome']))

In [95]:
list(set(df_4['parametro']))

['Coliformes totais',
 'pH',
 'Dióxido de Cloro (mg/L)',
 'Cloro residual livre (mg/L)',
 'Fluoreto (mg/L)',
 'Cor (uH)',
 'Escherichia coli',
 'Cloro residual combinado (mg/L)',
 'Turbidez (uT)',
 'Bactérias Heterotróficas']

In [96]:
df_4['analise_realizada'] = df_4['analise_realizada'].astype(str).str.replace('_', ' ')
df_4['analise_realizada'] = df_4['analise_realizada'].astype(str).str.replace('LABORATORIO', 'Laboratório')
df_4['analise_realizada'] = df_4['analise_realizada'].astype(str).str.replace('Laboratorio', 'Laboratório')
df_4['analise_realizada'] = df_4['analise_realizada'].str.title()
list(set(df_4['analise_realizada']))

['Em Laboratório', 'Em Campo', 'None']

In [97]:
# Multiplos Valores
#list(set(df_4['providencia']))

In [98]:
df_4.info()
df_4.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3215727 entries, 0 to 3215726
Data columns (total 32 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   id_ibge                         int64         
 1   n_amostra                       object        
 2   coleta_motivo                   object        
 3   forma_abastecimento_tipo        object        
 4   forma_abastecimento_cod         object        
 5   forma_abastecimento_nome        object        
 6   instituicao_tipo                object        
 7   instituicao_sigla               object        
 8   instituicao_nome                object        
 9   instituicao_cnpj                object        
 10  escritorio_regional_local_nome  object        
 11  escritorio_regional_local_cnpj  object        
 12  eta_uta_nome                    object        
 13  ano_referencia                  int64         
 14  mes_referencia                  int64         
 15

,id_ibge,n_amostra,coleta_motivo,forma_abastecimento_tipo,forma_abastecimento_cod,forma_abastecimento_nome,instituicao_tipo,instituicao_sigla,instituicao_nome,instituicao_cnpj,...,area_categoria,area_nome,local_tipo,local_nome,latitude,longitude,parametro,analise_realizada,resultado,providencia
0,3500105,2160,Rotina,SAA,S350010000001,ADAMANTINA,Empresa Estadual,SABESP,COMPANHIA DE SANEAMENTO BASICO DO ESTADO DE SA...,43776517000180.0,...,Bairro,Jardim Dalphalo,None,None,NaN,NaN,Cor (uH),None,"2,00",None
1,3500105,1424,Rotina,SAA,S350010000001,ADAMANTINA,Empresa Estadual,SABESP,COMPANHIA DE SANEAMENTO BASICO DO ESTADO DE SA...,43776517000180.0,...,Bairro,Jardim Tipuanas,None,None,NaN,NaN,pH,None,"7,60",None
2,3500105,3740,Rotina,SAA,S350010000001,ADAMANTINA,Empresa Estadual,SABESP,COMPANHIA DE SANEAMENTO BASICO DO ESTADO DE SA...,43776517000180.0,...,Bairro,Parque Jaraguá,None,None,NaN,NaN,Cloro residual livre (mg/L),Em Campo,"0,80",None
3,3500105,3251,Rotina,SAA,S350010000001,ADAMANTINA,Empresa Estadual,SABESP,COMPANHIA DE SANEAMENTO BASICO DO ESTADO DE SA...,43776517000180.0,...,Bairro,Parque Residencial Itamaraty,None,None,NaN,NaN,Turbidez (uT),Em Laboratório,"0,30",None
4,3500105,1423,Rotina,SAA,S350010000001,ADAMANTINA,Empresa Estadual,SABESP,COMPANHIA DE SANEAMENTO BASICO DO ESTADO DE SA...,43776517000180.0,...,Bairro,Jardim Aeroporto,None,None,NaN,NaN,Turbidez (uT),Em Laboratório,"0,20",None


<br>

### Instituições e Formas de Abastecimento

In [ ]:
list_cols = [
    'forma_abastecimento_cod',
    'forma_abastecimento_tipo',
    'forma_abastecimento_nome',
    'instituicao_tipo',
    'instituicao_sigla',
    'instituicao_nome',
    'instituicao_cnpj',
    'escritorio_regional_local_nome',
    'escritorio_regional_local_cnpj',
    'eta_uta_nome',
    'ano_referencia',
    'id_ibge',
]
df_4_2 = df_4[list_cols].drop_duplicates()
df_4_2.reset_index(drop=True, inplace=True)
df_4_2.info()
df_4_2.head()

<br>

## Demais Parametros

In [45]:
# Input
filename = os.path.join(input_path_parquet_partitioned, 'vigilancia', 'vigilancia_demais_parametros')

# Add Filter to Filename
filter_path = os.path.join('Uf={}'.format(estado), 'Código Ibge={}/*.parquet'.format(cod_ibge_ajustado))
filename = os.path.join(filename, filter_path)

# Read Dataframes
df = dd.read_parquet(
    filename,
    filters=[[('Uf', '==', estado)]],
)

# Calculate
df_5 = df.compute()
df_5 = rename_sisagua(df_5)
df_5.info()
df_5.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33079 entries, 0 to 2
Data columns (total 38 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   regiao_geografica               33079 non-null  object  
 1   regional_saude                  33079 non-null  object  
 2   municipio                       33079 non-null  object  
 3   n_amostra                       33079 non-null  object  
 4   coleta_motivo                   33079 non-null  object  
 5   forma_abastecimento_tipo        33079 non-null  object  
 6   forma_abastecimento_cod         33079 non-null  object  
 7   forma_abastecimento_nome        33079 non-null  object  
 8   instituicao_tipo                32704 non-null  object  
 9   instituicao_sigla               18752 non-null  object  
 10  instituicao_nome                32704 non-null  object  
 11  instituicao_cnpj                32704 non-null  object  
 12  escritorio_regional_lo

,regiao_geografica,regional_saude,municipio,n_amostra,coleta_motivo,forma_abastecimento_tipo,forma_abastecimento_cod,forma_abastecimento_nome,instituicao_tipo,instituicao_sigla,...,latitude,longitude,grupo_parametros,parametro,data_analise,lim_deteccao,lim_quantificacao,resultado,uf,id_ibge
0,SUDESTE,GVS XX - PIRACICABA,AGUAS DE SAO PEDRO,1536116,Rotina,SAA,S350060000002,ETA SABESP,Empresa Estadual,SABESP,...,None,None,Agrotóxicos,"Tebuconazol - VMP: 180,0 µg/L",03/07/2016,None,None,"0,2",SP,350060
1,SUDESTE,GVS XX - PIRACICABA,AGUAS DE SAO PEDRO,1536116,Rotina,SAA,S350060000002,ETA SABESP,Empresa Estadual,SABESP,...,None,None,Parâmetros Organolépticos,"1,4 Diclorobenzeno - VMP: 0,03 mg/L",11/07/2016,None,None,"0,001",SP,350060
2,SUDESTE,GVS XX - PIRACICABA,AGUAS DE SAO PEDRO,942716,Rotina,SAA,S350060000002,ETA SABESP,Empresa Estadual,SABESP,...,None,None,Produtos secundários de desinfecção,"2, 4, 6 Triclorofenol - VMP: 0,2 mg/L",12/04/2016,None,None,"0,01",SP,350060
3,SUDESTE,GVS XX - PIRACICABA,AGUAS DE SAO PEDRO,1536116,Rotina,SAA,S350060000002,ETA SABESP,Empresa Estadual,SABESP,...,None,None,Parâmetros Organolépticos,"Xilenos - VMP: 0,3 mg/L",11/07/2016,None,None,"0,001",SP,350060
4,SUDESTE,GVS XX - PIRACICABA,AGUAS DE SAO PEDRO,1536116,Rotina,SAA,S350060000002,ETA SABESP,Empresa Estadual,SABESP,...,None,None,Agrotóxicos,"Clordano - VMP: 0,2 µg/L",04/07/2016,None,None,"0,01",SP,350060


In [46]:
list(df_5.columns)

['regiao_geografica',
 'regional_saude',
 'municipio',
 'n_amostra',
 'coleta_motivo',
 'forma_abastecimento_tipo',
 'forma_abastecimento_cod',
 'forma_abastecimento_nome',
 'instituicao_tipo',
 'instituicao_sigla',
 'instituicao_nome',
 'instituicao_cnpj',
 'escritorio_regional_local_nome',
 'escritorio_regional_local_cnpj',
 'eta_uta_nome',
 'ano_referencia',
 'mes_referencia',
 'data_coleta',
 'data_laudo',
 'data_registro_sisagua',
 'coleta_procedencia',
 'coleta_ponto',
 'local_descricao',
 'zona',
 'area_categoria',
 'area_nome',
 'local_tipo',
 'local_nome',
 'latitude',
 'longitude',
 'grupo_parametros',
 'parametro',
 'data_analise',
 'lim_deteccao',
 'lim_quantificacao',
 'resultado',
 'uf',
 'id_ibge']

In [47]:
df_5.drop(
    [
        'regiao_geografica',
        'regional_saude',
        'municipio',
        # 'n_amostra',
        # 'coleta_motivo',
        # 'forma_abastecimento_tipo',
        # 'forma_abastecimento_cod',
        # 'forma_abastecimento_nome',
        'instituicao_tipo',
        'instituicao_sigla',
        'instituicao_nome',
        'instituicao_cnpj',
        'escritorio_regional_local_nome',
        'escritorio_regional_local_cnpj',
        'eta_uta_nome',
        # 'ano_referencia',
        # 'mes_referencia',
        # 'data_coleta',
        # 'data_laudo',
        # 'data_registro_sisagua',
        # 'coleta_procedencia',
        # 'coleta_ponto',
        # 'local_descricao',
        # 'zona',
        # 'area_categoria',
        # 'area_nome',
        # 'local_tipo',
        # 'local_nome',
        # 'latitude',
        # 'longitude',
        # 'grupo_parametros',
        # 'parametro',
        # 'data_analise',
        # 'lim_deteccao',
        # 'lim_quantificacao',
        # 'resultado',
        'uf',
        # 'id_ibge'
    ],
    inplace=True,
    axis=1,
    errors='ignore',    
)

In [48]:
df_5['id_ibge'] = df_5['id_ibge'].astype(int)
df_5 = convert_6d_to_7d(df_5)

In [49]:
df_5['ano_referencia'] = df_5['ano_referencia'].astype(int)
set(df_5['ano_referencia'])

{2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021}

In [50]:
df_5['mes_referencia'] = df_5['mes_referencia'].astype(int)
set(df_5['mes_referencia'])

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

In [51]:
df_5['data_coleta'] = pd.to_datetime(df_5['data_coleta'])
df_5['data_laudo'] = pd.to_datetime(df_5['data_laudo'])
df_5['data_registro_sisagua'] = pd.to_datetime(df_5['data_registro_sisagua'])

In [57]:
df_5['coleta_procedencia'] = df_5['coleta_procedencia'].str.title()
set(df_5['coleta_procedencia'])

{'Estação De Tratamento De Água',
 'Intra-Domiciliar / Intra-Predial',
 'Ponto De Captação(Água Superficial Ou Subterrânea)',
 'Sistema De Distribuição',
 'Solução Alternativa'}

In [58]:
df_5['coleta_ponto'] = df_5['coleta_ponto'].str.title()
set(df_5['coleta_ponto'])

{'Bebedouro',
 'Cavalete/Hidrômetro',
 None,
 'Pós-Filtração/Pré-Desinfecção',
 'Reservatório De Distribuição',
 'Reservatório De Água',
 'Saída De Tratamento/Pós-Desinfecção',
 'Torneira Antes Da Reservação',
 'Torneira Após A Reservação',
 'Água Pós-Filtração/Pré-Desinfecção'}

In [59]:
set(df_5['parametro'])

{'1,1 Dicloroeteno - VMP: 30,0 µg/L',
 '1,2 Diclorobenzeno - VMP: 0,01 mg/L',
 '1,2 Dicloroetano - VMP: 10,0 µg/L',
 '1,2 Dicloroeteno (cis + trans) - VMP: 50,0 µg/L',
 '1,4 Diclorobenzeno - VMP: 0,03 mg/L',
 '2, 4, 6 Triclorofenol - VMP: 0,2 mg/L',
 '2,4 D + 2,4,5 T - VMP: 30,0 µg/L',
 'Acrilamida - VMP: 0,5 µg/L',
 'Alaclor - VMP: 20,0 µg/L',
 'Aldicarbe + Aldicarbesulfona + Aldicarbesulfóxido - VMP: 10,0 µg/L',
 'Aldrin + Dieldrin - VMP: 0,03 µg/L',
 'Alumínio - VMP: 0,2 mg/L',
 'Amônia (como NH3) - VMP: 1,5 mg/L',
 'Antimônio - VMP: 0,005 mg/L',
 'Arsênio - VMP: 0,01 mg/L',
 'Atividade alfa total - VMP: 0,5 Bq/L',
 'Atividade beta total - VMP: 1,0 Bq/L',
 'Atrazina - VMP: 2,0 µg/L',
 'Benzeno - VMP: 5,0 µg/L',
 'Benzo(a)pireno - VMP: 0,7 µg/L',
 'Bromato - VMP: 0,01 mg/L',
 'Bário - VMP: 0,7 mg/L',
 'Carbendazim + benomil - VMP: 120,0 µg/L',
 'Carbofurano - VMP: 7,0 µg/L',
 'Chumbo - VMP: 0,01 mg/L',
 'Cianeto - VMP: 0,07 mg/L',
 'Clordano - VMP: 0,2 µg/L',
 'Cloreto - VMP: 250,0 m

In [60]:
df_5.loc[:, 'resultado'] = df_5['resultado'].astype(str).str.replace(',', '.')
df_5.loc[:, 'resultado'] = pd.to_numeric(
    df_5['resultado'],
    errors='coerce'
)
# TODO: Ajustar correções dos resultados
#set(df_5['resultado'])

In [61]:
df_5 = df_5.dropna(axis=0, subset=['resultado'])

In [62]:
df_5.info()
df_5.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30768 entries, 0 to 33078
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id_ibge                   30768 non-null  int64         
 1   n_amostra                 30768 non-null  object        
 2   coleta_motivo             30768 non-null  object        
 3   forma_abastecimento_tipo  30768 non-null  object        
 4   forma_abastecimento_cod   30768 non-null  object        
 5   forma_abastecimento_nome  30768 non-null  object        
 6   ano_referencia            30768 non-null  int64         
 7   mes_referencia            30768 non-null  int64         
 8   data_coleta               30768 non-null  datetime64[ns]
 9   data_laudo                30768 non-null  datetime64[ns]
 10  data_registro_sisagua     30768 non-null  datetime64[ns]
 11  coleta_procedencia        30768 non-null  object        
 12  coleta_ponto      

,id_ibge,n_amostra,coleta_motivo,forma_abastecimento_tipo,forma_abastecimento_cod,forma_abastecimento_nome,ano_referencia,mes_referencia,data_coleta,data_laudo,...,local_tipo,local_nome,latitude,longitude,grupo_parametros,parametro,data_analise,lim_deteccao,lim_quantificacao,resultado
0,3500600,1536116,Rotina,SAA,S350060000002,ETA SABESP,2016,6,2016-06-14,2016-08-02,...,None,None,None,None,Agrotóxicos,"Tebuconazol - VMP: 180,0 µg/L",03/07/2016,None,None,0.200
1,3500600,1536116,Rotina,SAA,S350060000002,ETA SABESP,2016,6,2016-06-14,2016-08-02,...,None,None,None,None,Parâmetros Organolépticos,"1,4 Diclorobenzeno - VMP: 0,03 mg/L",11/07/2016,None,None,0.001
2,3500600,942716,Rotina,SAA,S350060000002,ETA SABESP,2016,4,2016-04-05,2016-08-05,...,None,None,None,None,Produtos secundários de desinfecção,"2, 4, 6 Triclorofenol - VMP: 0,2 mg/L",12/04/2016,None,None,0.010
3,3500600,1536116,Rotina,SAA,S350060000002,ETA SABESP,2016,6,2016-06-14,2016-08-02,...,None,None,None,None,Parâmetros Organolépticos,"Xilenos - VMP: 0,3 mg/L",11/07/2016,None,None,0.001
4,3500600,1536116,Rotina,SAA,S350060000002,ETA SABESP,2016,6,2016-06-14,2016-08-02,...,None,None,None,None,Agrotóxicos,"Clordano - VMP: 0,2 µg/L",04/07/2016,None,None,0.010


<br>

## Cianobactérias Cianotoxinas

In [ ]:
# Input
filename = os.path.join(input_path_parquet_partitioned, 'vigilancia', 'vigilancia_cianobacterias_cianotoxinas')

# Add Filter to Filename
filter_path = os.path.join('Uf={}'.format(estado), 'Código Ibge={}/*.parquet'.format(cod_ibge_ajustado))
filename = os.path.join(filename, filter_path)

# Read Dataframes
df = dd.read_parquet(
    filename,
    filters=[[('Uf', '==', estado)]],
)

# Calculate
df_6 = df.compute()
df_6 = rename_sisagua(df_6)
df_6.info()
df_6.head()

In [ ]:
list(df_6.columns)

In [ ]:
df_6.drop(
    [
        'regiao_geografica',
        'regional_saude',
        'municipio',
        # 'n_amostra',
        # 'coleta_motivo',
        'forma_abastecimento_tipo',
        # 'forma_abastecimento_cod',
        'forma_abastecimento_nome',
        'instituicao_tipo',
        'Sigla Instituição',
        'instituicao_nome',
        'instituicao_cnpj',
        'escritorio_regional_local_nome',
        'escritorio_regional_local_cnpj',
        'eta_uta_nome',
        # 'ano_referencia',
        # 'mes_referencia',
        # 'data_coleta',
        # 'data_laudo',
        # 'data_registro_sisagua',
        # 'coleta_procedencia',
        # 'coleta_ponto',
        # 'local_descricao',
        # 'zona',
        # 'area_categoria',
        # 'area_nome',
        # 'local_tipo',
        # 'local_nome',
        # 'latitude',
        # 'longitude',
        # 'grupo',
        # 'parametro',
        # 'resultado',
        'uf',
        # 'id_ibge'
    ],
    inplace=True,
    axis=1,
    errors='ignore',    
)

In [ ]:
df_6['ano_referencia'] = df_6['ano_referencia'].astype(int)
set(df_6['ano_referencia'])

In [ ]:
df_6['mes_referencia'] = df_6['mes_referencia'].astype(int)
set(df_6['mes_referencia'])

In [ ]:
df_6['data_coleta'] = pd.to_datetime(df_6['data_coleta'])
df_6['data_laudo'] = pd.to_datetime(df_6['data_laudo'])
df_6['data_registro_sisagua'] = pd.to_datetime(df_6['data_registro_sisagua'])

In [ ]:
df_6['coleta_procedencia'] = df_6['coleta_procedencia'].str.title()
set(df_6['coleta_procedencia'])

In [ ]:
df_6['coleta_ponto'] = df_6['coleta_ponto'].str.title()
set(df_6['coleta_ponto'])

In [ ]:
df_6['local_descricao'] = df_6['local_descricao'].str.title()
set(df_6['local_descricao'])

In [ ]:
set(df_6['zona'])

In [ ]:
set(df_6['area_categoria'])

In [ ]:
df_6['area_nome'] = df_6['area_nome'].str.title()
set(df_6['area_nome'])

In [ ]:
set(df_6['local_tipo'])

In [ ]:
set(df_6['local_nome'])

In [ ]:
set(df_6['latitude'])

In [ ]:
set(df_6['longitude'])

In [ ]:
set(df_6['grupo'])

In [ ]:
set(df_6['parametro'])

In [ ]:
df_6['resultado'] = df_6['resultado'].astype(str).str.replace(',', '.')
df_6['resultado'] = pd.to_numeric(df_6['resultado'], errors='coerce')
set(df_6['resultado'])

In [ ]:
df_6.info()
df_6.head()

<br>

# Controle

<br>

## Mensal Parametros Basicos

In [ ]:
# Input
filename = os.path.join(input_path_parquet_partitioned, 'controle', 'controle_mensal_parametros_basicos_*')

# Add Filter to Filename
filter_path = os.path.join('Uf={}'.format(estado), 'Código Ibge={}/*.parquet'.format(cod_ibge_ajustado))
filename = os.path.join(filename, filter_path)

# Read Dataframes
df = dd.read_parquet(
    filename,
    filters=[[('Uf', '==', estado)]],
)

# Calculate
df_7 = df.compute()
df_7 = rename_sisagua(df_7)
df_7.info()
df_7.head()

In [ ]:
#del df_7

In [ ]:
list(df_7.columns)

In [ ]:
df_7.drop(
    [
        'regiao_geografica',
        'regional_saude',
        'municipio',
        'instituicao_tipo',
        'instituicao_sigla',
        'instituicao_nome',
        'instituicao_cnpj',
        'escritorio_regional_local_nome',
        'escritorio_regional_local_cnpj',
        # 'forma_abastecimento_tipo',
        'forma_abastecimento_cod',
        # 'forma_abastecimento_nome',
        'eta_uta_nome',
        'tipo_filtracao',
        # 'ano_referencia',
        # 'mes_referencia',
        # 'pto_monitoramento',
        # 'parametro',
        # 'campo',
        # 'resultado',
        'uf',
        # 'id_ibge'
    ],
    inplace=True,
    axis=1,
    errors='ignore',    
)

In [ ]:
df_7['id_ibge'] = df_7['id_ibge'].astype(int)
# df_7 = convert_6d_to_7d(df_7) # Dá erro!

In [ ]:
df_7['ano_referencia'] = df_7['ano_referencia'].astype(int)
set(df_7['ano_referencia'])

In [ ]:
df_7['mes_referencia'] = df_7['mes_referencia'].astype(int)
set(df_7['mes_referencia'])

In [ ]:
df_7['pto_monitoramento'] = df_7['pto_monitoramento'].str.title()
set(df_7['pto_monitoramento'])

In [ ]:
set(df_7['parametro'])

In [ ]:
set(df_7['campo'])

In [ ]:
# Passei para resultado
df_7['resultado'] = df_7['resultado'].astype(str).str.replace(',', '.')
df_7['resultado'] = pd.to_numeric(df_7['resultado'], errors='coerce')
list(set(df_7['resultado']))[0:10]

In [ ]:
df_7.info()
df_7.head()

<br>

## Mensal Demais Parametros

In [ ]:
# Input
filename = os.path.join(input_path_parquet_partitioned, 'controle', 'controle_mensal_demais_parametros')

# Add Filter to Filename
filter_path = os.path.join('Uf={}'.format(estado), 'Código Ibge={}/*.parquet'.format(cod_ibge_ajustado))
filename = os.path.join(filename, filter_path)

# Read Dataframes
df = dd.read_parquet(
    filename,
    filters=[[('Uf', '==', estado)]],
)

# Calculate
df_8 = df.compute()
df_8 = rename_sisagua(df_8)
df_8.info()
df_8.head()

In [ ]:
list(df_8.columns)

In [ ]:
df_8.drop(
    [
        'regiao_geografica',
        'regional_saude',
        'municipio',
        'instituicao_tipo',
        'instituicao_sigla',
        'instituicao_nome',
        'instituicao_cnpj',
        'escritorio_regional_local_nome',
        'escritorio_regional_local_cnpj',
        'forma_abastecimento_tipo',
        #'forma_abastecimento_cod',
        'forma_abastecimento_nome',
        'eta_uta_nome',
        # 'ano_referencia',
        # 'mes_referencia',
        # 'data_registro',
        # 'data_preenchimento_rel_mensal',
        # 'data_coleta',
        # 'captacao_tipo',
        # 'manancial_superficial_categoria',
        # 'manancial_superficial_nome',
        # 'captacao_subterranea_categoria',
        # 'captacao_subterranea_nome',
        # 'parametro',
        # 'unidade',
        # 'resultado',
        'uf',
        # 'id_ibge',
    ],
    inplace=True,
    axis=1,
    errors='ignore',    
)

In [ ]:
df_8['ano_referencia'] = df_8['ano_referencia'].astype(int)
set(df_8['ano_referencia'])

In [ ]:
df_8['mes_referencia'] = df_8['mes_referencia'].astype(int)
set(df_8['mes_referencia'])

In [ ]:
df_8['data_registro'] = pd.to_datetime(df_8['data_registro'])
df_8['data_coleta'] = pd.to_datetime(df_8['data_coleta'])
df_8['data_preenchimento_rel_mensal'] = pd.to_datetime(df_8['data_preenchimento_rel_mensal'])

In [ ]:
df_8['captacao_tipo'] = df_8['captacao_tipo'].str.title()
set(df_8['captacao_tipo'])

In [ ]:
df_8['manancial_superficial_categoria'] = df_8['manancial_superficial_categoria'].str.title()
set(df_8['manancial_superficial_categoria'])

In [ ]:
df_8['captacao_subterranea_categoria'] = df_8['captacao_subterranea_categoria'].str.title()
set(df_8['captacao_subterranea_categoria'])

In [ ]:
set(df_8['parametro'])

In [ ]:
set(df_8['unidade'])

In [ ]:
list(set(df_8['resultado']))

In [ ]:
df_8['resultado'] = df_8['resultado'].astype(str).str.replace(',', '.')
df_8['resultado'] = pd.to_numeric(df_8['resultado'], errors='coerce')

In [ ]:
#.sort()
#a = [x.replace(',', '.') for x in a if x is not None]
#a

In [ ]:
df_8.info()
df_8.head()

<br>

## Mensal Amostras Fora Padrão

In [ ]:
# Input
filename = os.path.join(input_path_parquet_partitioned, 'controle', 'controle_mensal_amostras_fora_padrao')

# Add Filter to Filename
filter_path = os.path.join('Uf={}'.format(estado), 'Código Ibge={}/*.parquet'.format(cod_ibge_ajustado))
filename = os.path.join(filename, filter_path)

# Read Dataframes
df = dd.read_parquet(
    filename,
    filters=[[('Uf', '==', estado)]],
)

# Calculate
df_9 = df.compute()
df_9 = rename_sisagua(df_9)
df_9.info()
df_9.head()

In [ ]:
list(df_9.columns)

In [ ]:
df_9.drop(
    [        
        'regiao_geografica',
        'regional_saude',
        'municipio',
        'instituicao_tipo',
        'instituicao_sigla',
        'instituicao_nome',
        'instituicao_cnpj',
        'escritorio_regional_local_nome',
        'escritorio_regional_local_cnpj',
        'forma_abastecimento_tipo',
        # 'forma_abastecimento_cod',
        'forma_abastecimento_nome',
        # 'ano_referencia',
        # 'mes_referencia',
        # 'data_registro',
        # 'data_preenchimento_rel_mensal',
        # 'data_coleta',
        # 'pto_monitoramento',
        # 'parametro',
        # 'zona',
        # 'area_categoria',
        # 'area_nome',
        # 'local_tipo',
        # 'local_nome',
        # 'endereco',
        # 'latitude',
        # 'longitude',
        # 'resultado',
        # 'providencia_controle',
        'uf',
        # 'id_ibge'
    ],
    inplace=True,
    axis=1,
    errors='ignore',
)

In [ ]:
df_9.info()
df_9.head()

<br>

## Mensal Infraestrutura Operacionais

In [ ]:
# Input
filename = os.path.join(input_path_parquet_partitioned, 'controle', 'controle_mensal_infraestrutura_operacionais')

# Add Filter to Filename
filter_path = os.path.join('Uf={}'.format(estado), 'Código Ibge={}/*.parquet'.format(cod_ibge_ajustado))
filename = os.path.join(filename, filter_path)

# Read Dataframes
df = dd.read_parquet(
    filename,
    filters=[[('Uf', '==', estado)]],
)

# Calculate
df_10 = df.compute()
df_10 = rename_sisagua(df_10)
df_10.info()
df_10.head()

In [ ]:
list(df_10.columns)

In [ ]:
df_10.drop(
    [        
        'regiao_geografica',
        'regional_saude',
        'municipio',
        'instituicao_tipo',
        'instituicao_sigla',
        'instituicao_nome',
        'instituicao_cnpj',
        'escritorio_regional_local_nome',
        'escritorio_regional_local_cnpj',
        'forma_abastecimento_tipo',
        # 'forma_abastecimento_cod',
        'forma_abastecimento_nome',
        # 'ano_referencia',
        # 'mes_referencia',
        # 'data_registro',
        # 'data_preenchimento_rel_mensal',
        # 'zona',
        # 'area_categoria',
        # 'area_nome',
        # 'local_tipo',
        # 'local_nome',
        # 'n_reparos_rede',
        # 'n_eventos_intermitencia',
        # 'n_eventos_falta_agua',
        # 'n_reclamacao_cor',
        # 'n_reclamacao_gosto_odor',
        'uf',
        # 'id_ibge'
    ],
    inplace=True,
    axis=1,
    errors='ignore',
)

In [ ]:
df_10['data_registro'] = pd.to_datetime(df_10['data_registro'])
df_10['data_preenchimento_rel_mensal'] = pd.to_datetime(df_10['data_preenchimento_rel_mensal'])

In [ ]:
df_10['ano_referencia'] = df_10['ano_referencia'].astype(int)
df_10['mes_referencia'] = df_10['mes_referencia'].astype(int)

In [ ]:
cols = [
    'n_reparos_rede',
    'n_eventos_intermitencia',
    'n_eventos_falta_agua',
    'n_reclamacao_cor',
    'n_reclamacao_gosto_odor',
]

for col in cols:
    df_10[col].fillna(-1, inplace=True)
    df_10[col] = df_10[col].astype(int)

df_10.reset_index(drop=True, inplace=True)

In [ ]:
df_10.info()
df_10.head()

<br>

## Semestral

In [ ]:
# Input
filename = os.path.join(input_path_parquet_partitioned, 'controle', 'controle_semestral_*')

# Add Filter to Filename
filter_path = os.path.join('Uf={}'.format(estado), 'Código Ibge={}/*.parquet'.format(cod_ibge_ajustado))
filename = os.path.join(filename, filter_path)

# Read Dataframes
df = dd.read_parquet(
    filename,
    filters=[[('Uf', '==', estado)]],
)

# Calculate
df_11 = df.compute()
df_11 = rename_sisagua(df_11)
df_11.info()
df_11.head()

In [ ]:
list(df_11.columns)

In [ ]:
df_11.drop(
    [
        'regiao_geografica',
        'regional_saude',
        'municipio',
        'instituicao_tipo',
        'instituicao_sigla',
        'instituicao_nome',
        'instituicao_cnpj',
        'escritorio_regional_local_nome',
        'escritorio_regional_local_cnpj',
        'forma_abastecimento_tipo',
        # 'forma_abastecimento_cod',
        'forma_abastecimento_nome',
        'eta_uta_nome',
        # 'ano_referencia',
        # 'semestre_referencia',
        # 'data_registro',
        # 'data_preenchimento_rel_semestral',
        # 'data_coleta',
        # 'data_analise',
        # 'pto_monitoramento',
        # 'grupo_parametros',
        # 'parametro',
        # 'lim_deteccao',
        # 'lim_quantificacao',
        # 'resultado',
        'uf',
        # 'id_ibge'
    ],
    inplace=True,
    axis=1,
    errors='ignore',    
)

In [ ]:
df_11['ano_referencia'] = df_11['ano_referencia'].astype(int)
set(df_11['ano_referencia'])

In [ ]:
df_11['semestre_referencia'] = df_11['semestre_referencia'].astype(int)
set(df_11['semestre_referencia'])

In [ ]:
df_11['data_registro'] = pd.to_datetime(df_11['data_registro'])
df_11['data_preenchimento_rel_semestral'] = pd.to_datetime(df_11['data_preenchimento_rel_semestral'])

In [ ]:
df_11['data_coleta'] = pd.to_datetime(df_11['data_coleta'], format='%d/%m/%Y')
df_11['data_analise'] = pd.to_datetime(df_11['data_analise'], format='%d/%m/%Y')

In [ ]:
df_11['resultado'] = df_11['resultado'].astype(str).str.replace(',', '.')
df_11['resultado'] = pd.to_numeric(df_11['resultado'], errors='coerce')
set(df_11['resultado'])

In [ ]:
df_11.info()
df_11.head()

<br>

# End

In [ ]:
cluster.close()
time.sleep(1)
client.close()

<br>

# Análises

<br>

## Cadastro

<br>

### Municípios

In [ ]:
df_mun = pd.concat([df_1_1, df_2_1, df_3_1]).drop_duplicates()
df_mun.reset_index(drop=True, inplace=True)
df_mun.info()
df_mun.head()

In [ ]:
df_mun = convert_6d_to_7d(df_mun, del_municipios_name=False)
df_mun.info()
df_mun.head()

In [ ]:
# Ajusta o erro único em Presidente Venceslau
df_mun['regional_saude'] = df_mun['regional_saude'].replace(' -PRESIDENTE', ' - PRESIDENTE', regex=True)

# Expande Colunas
df_mun[['regional_saude_cod', 'regional_saude_sede']] = df_mun['regional_saude'].str.split(' - ', 1, expand=True)
df_mun['regional_saude_sede'] = df_mun['regional_saude_sede'].str.title()
df_mun['regional_saude_sede'] = df_mun['regional_saude_sede'].str.strip()
df_mun.info()
df_mun.head()

In [ ]:
df_mun['regional_saude_sede'] = df_mun['regional_saude_sede'].replace(
    {
        #'Botucatu E Sgvs Xvi Avaré': 'Botucatu (SGVS XVI Avaré)',
        #'Botucatu e Sgvs Xvi Avaré': 'Botucatu (SGVS XVI Avaré)',
        'Botucatu e Sgvs Xvi Avaré': 'Botucatu',
        'Botucatu E Sgvs Xvi Avaré': 'Botucatu',
        ' Das ': ' das ',
        ' Da ': ' da ',
        ' Dos ': ' dos ',
        ' Do ': ' do ',
        ' E ': ' e ',
    },
    regex=True,
)
df_mun.info()
df_mun.head()

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_mun.to_sql(
        'cadastro_municipios',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        #chunksize=500,
        method=psql_insert_copy,
    )

<br>

### Instituições e Formas de Abastecimento

In [ ]:
#df_2_2.info()
#df_4_2.info()

In [ ]:
df_3_2.info()

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_3_2.to_sql(
        'cadastro_formas_instituicoes',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        #chunksize=500,
        method=psql_insert_copy,
    )

<br>

### Captação

In [ ]:
df_1_3.info()
df_1_3.head()

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_1_3.to_sql(
        'cadastro_captacao',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        #chunksize=500,
        method=psql_insert_copy,
    )

<br>

### População

In [ ]:
df_2_3.info()
df_2_3.head()

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_2_3.to_sql(
        'cadastro_populacao',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        #chunksize=500,
        method=psql_insert_copy,
    )

<br>

### Tratamento

In [ ]:
df_3_3.info()
df_3_3.head()

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_3_3.to_sql(
        'cadastro_tratamento',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        #chunksize=500,
        method=psql_insert_copy,
    )

<br>

## Vigilância

<br>

### Parâmetros Básicos

Separar por paraâmetros e corrigor resultados

In [99]:
list_pars = list(set(df_4['parametro']))
list_pars.sort()
list_pars

['Bactérias Heterotróficas',
 'Cloro residual combinado (mg/L)',
 'Cloro residual livre (mg/L)',
 'Coliformes totais',
 'Cor (uH)',
 'Dióxido de Cloro (mg/L)',
 'Escherichia coli',
 'Fluoreto (mg/L)',
 'Turbidez (uT)',
 'pH']

<br>

#### Bactérias Heterotróficas

In [ ]:
df_4_bacte = df_4[df_4['parametro'] == 'Bactérias Heterotróficas'].copy()
df_4_bacte.reset_index(drop=True, inplace=True)

df_4_bacte.info()
df_4_bacte.head()

In [ ]:
#set(df_4_bacte['resultado'])

In [ ]:
# Notei que nos resultados existem alguns valores com espaços (por exemplo, "> 500").
# Splitei a coluna

df_4_bacte[['a','b']] = df_4_bacte['resultado'].str.split(' ', 1, expand=True)

In [ ]:
# Criei as colunas a e b.
# Quando ambas não são nulas, crie a coluna "sinal" e coloquei o ">"

mask = (df_4_bacte['a'].notnull()) & (df_4_bacte['b'].notnull())
df_4_bacte.loc[mask, 'sinal'] = df_4_bacte['a']
df_4_bacte.loc[mask, 'resultado'] = df_4_bacte['b']

In [ ]:
# Depois salvei o número, efetivamente

mask = (df_4_bacte['a'].notnull()) & (df_4_bacte['b'].isnull())
df_4_bacte.loc[mask, 'resultado'] = df_4_bacte['a']

In [ ]:
# Agora sim; converti tudo para float

df_4_bacte.loc[:, 'resultado'] = df_4_bacte['resultado'].astype(str).str.replace(',', '.')
df_4_bacte['resultado'] = pd.to_numeric(df_4_bacte['resultado'], errors='coerce')

In [ ]:
df_4_bacte.drop(['a', 'b'], inplace=True, axis=1, errors='ignore')

In [ ]:
df_4_bacte.info()
df_4_bacte.head()

In [ ]:
df_4_bacte = df_4_bacte.dropna(axis=0, subset=['resultado'])

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_4_bacte.to_sql(
        'vig_par_basico_bacterias',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        #chunksize=500,
        method=psql_insert_copy,
    )

In [ ]:
#df_4_bacte['resultado'].hist(bins=300)

In [ ]:
#list(set(df_4_bacte['resultado']))[0:1000]

In [ ]:
# TODO
# df_4['resultado'] = df_4['resultado'].astype(str).str.replace(',', '#')
# #df_4['resultado'] = df_4['resultado'].astype(str).str.replace('.', ',')
# df_4['resultado'] = df_4['resultado'].astype(str).str.replace('#', '.')
# df_4['resultado'] = pd.to_numeric(df_4['resultado'], errors='coerce')
# list(set(df_4['resultado']))[0:1000]

In [ ]:
# mask = (df_4_bacte['a'].notnull()) & (df_4_bacte['b'].notnull())
# df_4_bacte.loc[mask,[
#     'resultado',
#     'sinal',
#     #'bb'
# ]].drop_duplicates()

In [ ]:
# df_4_bacte.replace({'resultado': {
#     '0.000.00': 0,
#     '00.000.00': 0,
#     '5.000.00': 0,
#     'nan': 0}}, inplace=True)


# df_4_bacte['resultado'] = df_4_bacte['resultado'].astype(str).str.replace(',', '.')
# df_4_bacte['resultado'] = pd.to_numeric(df_4_bacte['resultado'])

In [ ]:
# # List

# df_4_bacte.loc[:,'resultado'] = pd.to_numeric(df_4_bacte['resultado'], errors='coerce')
# list_res = list(set(df_4_bacte['resultado']))
# list_res.sort()
# print(len(list_res))

# # Loop
# list_values = []
# for i in list_res:
#     try:
#         float(i)
#     except ValueError:
#         list_values.append(i)

# # Results
# len(list_values)
# list_values

In [ ]:
# #df_4[df_4['resultado'].isin(list_values)]
# df_temp = df_4[df_4['resultado'].isin(['< 500.0', '> 500.0', 'AUSENTE', 'NAO_REALIZADA', 'PRESENTE'])]
# df_temp

In [ ]:

# g = 9
# print(list_values[g])
# df_temp = df_4[df_4['resultado'] == list_values[g]]

# df_temp_2 = df_temp[df_temp['resultado'] == list_values[g]]

# print(len(df_temp_2))
# df_temp_2.head()

<br>

#### Cloro Residual Combinado (mg/L)

In [ ]:
df_4_cloco = df_4[df_4['parametro'] == 'Cloro residual combinado (mg/L)'].copy()
df_4_cloco.reset_index(drop=True, inplace=True)

df_4_cloco.info()
df_4_cloco.head()

In [ ]:
df_4_cloco.loc[:, 'resultado'] = df_4_cloco['resultado'].astype(str).str.replace(',', '.')
df_4_cloco.loc[:, 'resultado'] = pd.to_numeric(df_4_cloco['resultado'], errors='coerce')
set(df_4_cloco['resultado'])

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_4_cloco.to_sql(
        'vig_par_basico_cloro_combinado',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        chunksize=500,
        method=psql_insert_copy,
    )

<br>

#### Cloro Residual Livre (mg/L)

In [ ]:
df_4_cloli = df_4[df_4['parametro'] == 'Cloro residual livre (mg/L)'].copy()
df_4_cloli.reset_index(drop=True, inplace=True)

df_4_cloli.info()
df_4_cloli.head()

In [ ]:
set(df_4_cloli['resultado'])
df_4_cloli.loc[:, 'resultado'] = df_4_cloli['resultado'].astype(str).str.replace(',', '.')
df_4_cloli.loc[:, 'resultado'] = pd.to_numeric(df_4_cloli['resultado'], errors='coerce')

In [ ]:
df_4_cloli = df_4_cloli.dropna(axis=0, subset=['resultado'])

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_4_cloli.to_sql(
        'vig_par_basico_cloro_livre',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        chunksize=5000,
        method=psql_insert_copy,
    )

<br>

#### Coliformes Totais

- Resultados expressos em: Ausente, Presente

In [ ]:
df_4_colif = df_4[df_4['parametro'] == 'Coliformes totais'].copy()
df_4_colif.reset_index(drop=True, inplace=True)

df_4_colif.info()
df_4_colif.head()

In [ ]:
df_4_colif['resultado'] = df_4_colif['resultado'].str.title()
set(df_4_colif['resultado'])

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_4_colif.to_sql(
        'vig_par_basico_coliformes',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        chunksize=5000,
        method=psql_insert_copy,
    )

<br>

#### Cor (uH)

In [ ]:
df_4_cor = df_4[df_4['parametro'] == 'Cor (uH)'].copy()
df_4_cor.reset_index(drop=True, inplace=True)

df_4_cor.info()
df_4_cor.head()

In [ ]:
set(df_4_cor['resultado'])
df_4_cor.loc[:, 'resultado'] = df_4_cor['resultado'].astype(str).str.replace(',', '.')
df_4_cor.loc[:, 'resultado'] = pd.to_numeric(df_4_cor['resultado'], errors='coerce')

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_4_cor.to_sql(
        'vig_par_basico_cor',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        chunksize=5000,
        method=psql_insert_copy,
    )

<br>

#### Dióxido de Cloro (mg/L)

In [ ]:
df_4_dicl = df_4[df_4['parametro'] == 'Dióxido de Cloro (mg/L)'].copy()
df_4_dicl.reset_index(drop=True, inplace=True)

df_4_dicl.info()
df_4_dicl.head()

In [ ]:
set(df_4_dicl['resultado'])
df_4_dicl.loc[:, 'resultado'] = df_4_dicl['resultado'].astype(str).str.replace(',', '.')
df_4_dicl.loc[:, 'resultado'] = pd.to_numeric(df_4_dicl['resultado'], errors='coerce')

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_4_dicl.to_sql(
        'vig_par_basico_di_cloro',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        chunksize=5000,
        method=psql_insert_copy,
    )

<br>

#### Escherichia coli

In [ ]:
df_4_ecol = df_4[df_4['parametro'] == 'Escherichia coli'].copy()
df_4_ecol.reset_index(drop=True, inplace=True)

df_4_ecol.info()
df_4_ecol.head()

In [ ]:
df_4_ecol['resultado'] = df_4_ecol['resultado'].str.title()
set(df_4_ecol['resultado'])

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_4_ecol.to_sql(
        'vig_par_basico_e_coli',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        chunksize=5000,
        method=psql_insert_copy,
    )

<br>

#### Fluoreto (mg/L)

In [ ]:
df_4_fluo = df_4[df_4['parametro'] == 'Fluoreto (mg/L)'].copy()
df_4_fluo.reset_index(drop=True, inplace=True)

df_4_fluo.info()
df_4_fluo.head()

In [ ]:
df_4_fluo.loc[:, 'resultado'] = df_4_fluo['resultado'].astype(str).str.replace(',', '.')
df_4_fluo.loc[:, 'resultado'] = pd.to_numeric(df_4_fluo['resultado'], errors='coerce')
set(df_4_fluo['resultado'])

In [ ]:
df_4_fluo = df_4_fluo.dropna(axis=0, subset=['resultado'])

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_4_fluo.to_sql(
        'vig_par_basico_fluor',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        chunksize=5000,
        method=psql_insert_copy,
    )

<br>

#### Turbidez (uT)

In [100]:
df_4_turb = df_4[df_4['parametro'] == 'Turbidez (uT)'].copy()
df_4_turb.reset_index(drop=True, inplace=True)

df_4_turb.info()
df_4_turb.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488699 entries, 0 to 488698
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   id_ibge                         488699 non-null  int64         
 1   n_amostra                       488699 non-null  object        
 2   coleta_motivo                   488699 non-null  object        
 3   forma_abastecimento_tipo        488699 non-null  object        
 4   forma_abastecimento_cod         488699 non-null  object        
 5   forma_abastecimento_nome        488699 non-null  object        
 6   instituicao_tipo                487862 non-null  object        
 7   instituicao_sigla               259462 non-null  object        
 8   instituicao_nome                487862 non-null  object        
 9   instituicao_cnpj                487862 non-null  object        
 10  escritorio_regional_local_nome  259462 non-null  object 

,id_ibge,n_amostra,coleta_motivo,forma_abastecimento_tipo,forma_abastecimento_cod,forma_abastecimento_nome,instituicao_tipo,instituicao_sigla,instituicao_nome,instituicao_cnpj,...,area_categoria,area_nome,local_tipo,local_nome,latitude,longitude,parametro,analise_realizada,resultado,providencia
0,3500105,3251,Rotina,SAA,S350010000001,ADAMANTINA,Empresa Estadual,SABESP,COMPANHIA DE SANEAMENTO BASICO DO ESTADO DE SA...,43776517000180.0,...,Bairro,Parque Residencial Itamaraty,None,None,NaN,NaN,Turbidez (uT),Em Laboratório,"0,30",None
1,3500105,1423,Rotina,SAA,S350010000001,ADAMANTINA,Empresa Estadual,SABESP,COMPANHIA DE SANEAMENTO BASICO DO ESTADO DE SA...,43776517000180.0,...,Bairro,Jardim Aeroporto,None,None,NaN,NaN,Turbidez (uT),Em Laboratório,"0,20",None
2,3500105,364,Rotina,SAA,S350010000001,ADAMANTINA,Empresa Estadual,SABESP,COMPANHIA DE SANEAMENTO BASICO DO ESTADO DE SA...,43776517000180.0,...,Bairro,Jardim América,None,None,NaN,NaN,Turbidez (uT),Em Laboratório,"0,30",None
3,3500105,4085,Rotina,SAA,S350010000001,ADAMANTINA,Empresa Estadual,SABESP,COMPANHIA DE SANEAMENTO BASICO DO ESTADO DE SA...,43776517000180.0,...,Bairro,Parque Iguaçu,None,None,NaN,NaN,Turbidez (uT),Em Laboratório,"0,30",None
4,3500105,2159,Rotina,SAA,S350010000001,ADAMANTINA,Empresa Estadual,SABESP,COMPANHIA DE SANEAMENTO BASICO DO ESTADO DE SA...,43776517000180.0,...,Bairro,Conjunto Das Palmeiras,None,None,NaN,NaN,Turbidez (uT),Em Laboratório,"0,30",None


In [101]:
set(df_4_turb['resultado'])
df_4_turb.loc[:, 'resultado'] = df_4_turb['resultado'].astype(str).str.replace(',', '.')
df_4_turb.loc[:, 'resultado'] = pd.to_numeric(df_4_turb['resultado'], errors='coerce')

In [102]:
df_4_turb = df_4_turb.dropna(axis=0, subset=['resultado'])

In [104]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_4_turb.to_sql(
        'vig_par_basico_turbidez',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        chunksize=5000,
        method=psql_insert_copy,
    )

<br>

#### pH

In [ ]:
df_4_ph = df_4[df_4['parametro'] == 'pH'].copy()
df_4_ph.reset_index(drop=True, inplace=True)

df_4_ph.info()
df_4_ph.head()

In [ ]:
set(df_4_ph['resultado'])
df_4_ph.loc[:, 'resultado'] = df_4_ph['resultado'].astype(str).str.replace(',', '.')
df_4_ph.loc[:, 'resultado'] = pd.to_numeric(df_4_ph['resultado'], errors='coerce')

In [ ]:
df_4_ph = df_4_ph.dropna(axis=0, subset=['resultado'])

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_4_ph.to_sql(
        'vig_par_basico_ph',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        chunksize=5000,
        method=psql_insert_copy,
    )

<br>

### Demais Parâmetros

In [63]:
df_5.info()
df_5.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30768 entries, 0 to 33078
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id_ibge                   30768 non-null  int64         
 1   n_amostra                 30768 non-null  object        
 2   coleta_motivo             30768 non-null  object        
 3   forma_abastecimento_tipo  30768 non-null  object        
 4   forma_abastecimento_cod   30768 non-null  object        
 5   forma_abastecimento_nome  30768 non-null  object        
 6   ano_referencia            30768 non-null  int64         
 7   mes_referencia            30768 non-null  int64         
 8   data_coleta               30768 non-null  datetime64[ns]
 9   data_laudo                30768 non-null  datetime64[ns]
 10  data_registro_sisagua     30768 non-null  datetime64[ns]
 11  coleta_procedencia        30768 non-null  object        
 12  coleta_ponto      

,id_ibge,n_amostra,coleta_motivo,forma_abastecimento_tipo,forma_abastecimento_cod,forma_abastecimento_nome,ano_referencia,mes_referencia,data_coleta,data_laudo,...,local_tipo,local_nome,latitude,longitude,grupo_parametros,parametro,data_analise,lim_deteccao,lim_quantificacao,resultado
0,3500600,1536116,Rotina,SAA,S350060000002,ETA SABESP,2016,6,2016-06-14,2016-08-02,...,None,None,None,None,Agrotóxicos,"Tebuconazol - VMP: 180,0 µg/L",03/07/2016,None,None,0.200
1,3500600,1536116,Rotina,SAA,S350060000002,ETA SABESP,2016,6,2016-06-14,2016-08-02,...,None,None,None,None,Parâmetros Organolépticos,"1,4 Diclorobenzeno - VMP: 0,03 mg/L",11/07/2016,None,None,0.001
2,3500600,942716,Rotina,SAA,S350060000002,ETA SABESP,2016,4,2016-04-05,2016-08-05,...,None,None,None,None,Produtos secundários de desinfecção,"2, 4, 6 Triclorofenol - VMP: 0,2 mg/L",12/04/2016,None,None,0.010
3,3500600,1536116,Rotina,SAA,S350060000002,ETA SABESP,2016,6,2016-06-14,2016-08-02,...,None,None,None,None,Parâmetros Organolépticos,"Xilenos - VMP: 0,3 mg/L",11/07/2016,None,None,0.001
4,3500600,1536116,Rotina,SAA,S350060000002,ETA SABESP,2016,6,2016-06-14,2016-08-02,...,None,None,None,None,Agrotóxicos,"Clordano - VMP: 0,2 µg/L",04/07/2016,None,None,0.010


In [65]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_5.to_sql(
        'vig_demais_parametros',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        chunksize=5000,
        method=psql_insert_copy,
    )

<br>

### Cianobactérias Cianotoxinas

In [ ]:
df_6.info()
df_6.head()

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_6.to_sql(
        'vig_cianobacterias_cianotoxinas',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        chunksize=5000,
        method=psql_insert_copy,
    )

<br>

## Controle

<br>

### Parâmetros Básicos

<br>

#### Bactérias Heterotróficas

In [28]:
df_7_bacte = df_7[df_7['parametro'] == 'Bactérias Heterotróficas (UFC/mL)'].copy()
df_7_bacte.reset_index(drop=True, inplace=True)

In [29]:
df_7_bacte.info()
df_7_bacte.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456576 entries, 0 to 456575
Data columns (total 9 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   forma_abastecimento_tipo  456576 non-null  object 
 1   forma_abastecimento_nome  456576 non-null  object 
 2   ano_referencia            456576 non-null  int64  
 3   mes_referencia            456576 non-null  int64  
 4   pto_monitoramento         456576 non-null  object 
 5   parametro                 456576 non-null  object 
 6   campo                     456576 non-null  object 
 7   resultado                 456576 non-null  float64
 8   id_ibge                   456576 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 31.4+ MB


,forma_abastecimento_tipo,forma_abastecimento_nome,ano_referencia,mes_referencia,pto_monitoramento,parametro,campo,resultado,id_ibge
0,SAA,LAGOA SECA,2014,4,Sistema De Distribuição,Bactérias Heterotróficas (UFC/mL),Número de amostras analisadas,2.0,350010
1,SAA,LAGOA SECA,2014,4,Sistema De Distribuição,Bactérias Heterotróficas (UFC/mL),Número de resultados > 500 UFC/mL,0.0,350010
2,SAA,LAGOA SECA,2014,4,Sistema De Distribuição,Bactérias Heterotróficas (UFC/mL),Número de resultados < 500 UFC/mL,2.0,350010
3,SAA,LAGOA SECA,2014,5,Sistema De Distribuição,Bactérias Heterotróficas (UFC/mL),Número de amostras analisadas,2.0,350010
4,SAA,LAGOA SECA,2014,5,Sistema De Distribuição,Bactérias Heterotróficas (UFC/mL),Número de resultados > 500 UFC/mL,0.0,350010


In [30]:
df_7_bacte = convert_6d_to_7d(df_7_bacte)

In [31]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_7_bacte.to_sql(
        'con_basics_bacterias',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        chunksize=5000,
        method=psql_insert_copy,
    )

<br>

#### Cloro Residual Combinado (mg/L)

In [32]:
df_7_cloco = df_7[df_7['parametro'] == 'Cloro Residual Combinado (mg/L)'].copy()
df_7_cloco.reset_index(drop=True, inplace=True)

In [33]:
df_7_cloco.info()
df_7_cloco.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3824 entries, 0 to 3823
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   forma_abastecimento_tipo  3824 non-null   object 
 1   forma_abastecimento_nome  3824 non-null   object 
 2   ano_referencia            3824 non-null   int64  
 3   mes_referencia            3824 non-null   int64  
 4   pto_monitoramento         3824 non-null   object 
 5   parametro                 3824 non-null   object 
 6   campo                     3824 non-null   object 
 7   resultado                 3824 non-null   float64
 8   id_ibge                   3824 non-null   int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 269.0+ KB


,forma_abastecimento_tipo,forma_abastecimento_nome,ano_referencia,mes_referencia,pto_monitoramento,parametro,campo,resultado,id_ibge
0,SAA,S1 - CENTRAL,2014,2,Saída Do Tratamento,Cloro Residual Combinado (mg/L),Número de amostras analisadas,28.0,350460
1,SAA,S1 - CENTRAL,2014,2,Saída Do Tratamento,Cloro Residual Combinado (mg/L),Percentil 95,0.0,350460
2,SAA,S1 - CENTRAL,2014,2,Saída Do Tratamento,Cloro Residual Combinado (mg/L),"Número de dados > 4,0 mg/L",0.0,350460
3,SAA,S1 - CENTRAL,2014,2,Saída Do Tratamento,Cloro Residual Combinado (mg/L),"Número de dados >= 2,0 mg/L e <= 4,0 mg/L",0.0,350460
4,SAA,S1 - CENTRAL,2014,2,Saída Do Tratamento,Cloro Residual Combinado (mg/L),"Número de dados < 2,0 mg/L",28.0,350460


In [34]:
df_7_cloco = convert_6d_to_7d(df_7_cloco)

In [35]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_7_cloco.to_sql(
        'con_basics_cloro_combinado',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        chunksize=5000,
        method=psql_insert_copy,
    )

<br>

#### Cloro Residual Livre (mg/L)

In [ ]:
df_7_cloli = df_7[df_7['parametro'] == 'Cloro Residual Livre (mg/L)'].copy()
df_7_cloli.reset_index(drop=True, inplace=True)

In [ ]:
df_7_cloli.info()
df_7_cloli.head()

In [ ]:
df_7_cloli = convert_6d_to_7d(df_7_cloli)

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_7_cloli.to_sql(
        'con_basics_cloro_livre',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        chunksize=5000,
        method=psql_insert_copy,
    )

<br>

#### Coliformes Totais

In [ ]:
df_7_colif = df_7[df_7['parametro'] == 'Coliformes totais'].copy()
df_7_colif.reset_index(drop=True, inplace=True)

In [ ]:
df_7_colif.info()
df_7_colif.head()

In [ ]:
df_7_colif = convert_6d_to_7d(df_7_colif)

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_7_colif.to_sql(
        'con_basics_coliformes_totais',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        chunksize=5000,
        method=psql_insert_copy,
    )

<br>

#### Cor (uH)

In [ ]:
df_7_cor = df_7[df_7['parametro'] == 'Cor (uH)'].copy()
df_7_cor.reset_index(drop=True, inplace=True)

In [ ]:
df_7_cor.info()
df_7_cor.head()

In [ ]:
df_7_cor = convert_6d_to_7d(df_7_cor)

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_7_cor.to_sql(
        'con_basics_cor',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        chunksize=5000,
        method=psql_insert_copy,
    )

<br>

#### Dióxido de Cloro (mg/L)

In [ ]:
df_7_dicl = df_7[df_7['parametro'] == 'Dióxido de Cloro'].copy()
df_7_dicl.reset_index(drop=True, inplace=True)

In [ ]:
df_7_dicl.info()
df_7_dicl.head()

In [ ]:
df_7_dicl = convert_6d_to_7d(df_7_dicl)

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_7_dicl.to_sql(
        'con_basics_cloro_dioxido',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        chunksize=5000,
        method=psql_insert_copy,
    )

<br>

#### Escherichia coli

In [ ]:
df_7_ecol = df_7[df_7['parametro'] == 'Escherichia coli'].copy()
df_7_ecol.reset_index(drop=True, inplace=True)

In [ ]:
df_7_ecol.info()
df_7_ecol.head()

In [ ]:
df_7_ecol = convert_6d_to_7d(df_7_ecol)

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_7_ecol.to_sql(
        'con_basics_e_coli',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        chunksize=5000,
        method=psql_insert_copy,
    )

<br>

#### Fluoreto (mg/L)

In [ ]:
df_7_fluo = df_7[df_7['parametro'] == 'Fluoreto (mg/L)'].copy()
df_7_fluo.reset_index(drop=True, inplace=True)

In [ ]:
df_7_fluo.info()
df_7_fluo.head()

In [ ]:
df_7_fluo = convert_6d_to_7d(df_7_fluo)

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_7_fluo.to_sql(
        'con_basics_fluor',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        chunksize=5000,
        method=psql_insert_copy,
    )

<br>

#### Turbidez (uT)

In [ ]:
df_7_turb = df_7[df_7['parametro'] == 'Turbidez (uT)'].copy()
df_7_turb.reset_index(drop=True, inplace=True)

In [ ]:
df_7_turb.info()
df_7_turb.head()

In [ ]:
df_7_turb = convert_6d_to_7d(df_7_turb)

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_7_turb.to_sql(
        'con_basics_turbidez',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        chunksize=5000,
        method=psql_insert_copy,
    )

<br>

#### pH

In [ ]:
df_7_ph = df_7[df_7['parametro'] == 'pH'].copy()
df_7_ph.reset_index(drop=True, inplace=True)

In [ ]:
df_7_ph.info()
df_7_ph.head()

In [ ]:
df_7_ph = convert_6d_to_7d(df_7_ph)

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    df_7_ph.to_sql(
        'con_basics_ph',
        con=conn,
        if_exists='replace',
        schema='michelmetran/br_sisagua',
        index=False,
        chunksize=5000,
        method=psql_insert_copy,
    )

<br>

# SQL

In [ ]:
sql = '''
    DROP TABLE IF EXISTS "michelmetran/br_sisagua"."df_1_1";
'''

with engine.connect().execution_options(autocommit=True) as conn:
    query = conn.execute(text(sql))

In [ ]:
df_1_1.head()

In [ ]:
df_1_1.to_sql(
    'df_1_1',
    con = engine,
    if_exists='replace',
    schema='michelmetran/br_sisagua',
    index=False,
    chunksize=500,
    #dtype=dict_dtypes,
)

df

<br>

## dtypes

In [ ]:
def get_dtypes_sqlalchemy(df):
    dict_cols = df.dtypes.apply(lambda x: x.name).to_dict()
    for k,v in dict_cols.items():
        if dict_cols[k] == 'object':
            dict_cols[k] = Text
        if dict_cols[k] == 'int64':
            dict_cols[k] = Integer
        if dict_cols[k] == 'datetime64[ns]':
            dict_cols[k] = DateTime
        if dict_cols[k] == 'float64':
            dict_cols[k] = Float
        return dict_cols

In [ ]:
dict_dtypes = get_dtypes_sqlalchemy(df_1_1)
dict_dtypes

In [ ]:
dict_dtypes = {
    'id_ibge': sqlalchemy.sql.sqltypes.Integer,
    'municipio': sqlalchemy.sql.sqltypes.Text,
    'regional_saude': sqlalchemy.sql.sqltypes.Text,
}

dict_dtypes